In [1]:
# Load PDF file

from langchain_community.document_loaders import PyMuPDFLoader

my_cv_path = "/Users/rsukumar/Downloads/Sukumar_RAGHAVAN_CV.pdf"

loader = PyMuPDFLoader(my_cv_path)
pdf_loaded_data = loader.load()

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
import chromadb

# Adjust LLM parameters
llm_config = {
    "temperature": 0.7,
    "top_p": 0.9,
    "presence_penalty": 0.6,
    "frequency_penalty": 0.6,
}

# Adjust Vector client
chroma_db_persist_path = "chroma-db-job-coverletter"
vector_db_collection_name = "job-cv-vector-db-collection"
# Keeping option to switch between Ephemeral / Persistent chroma client, if wanted
# ephemeral_chroma_client = chromadb.EphemeralClient()
persistent_chroma_client = chromadb.PersistentClient(path=chroma_db_persist_path)

model_config = {
    "langchain_embeddings": HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5"),
    # "langchain_embeddings": OllamaEmbeddings(model="llama3.2"),
    "llm_instance": ChatOllama(model="llama3.2", config=llm_config),
    "vector_db_client": persistent_chroma_client,
}

In [10]:
# Store vector embeddings into the vector db for later use

from chromadb.utils.embedding_functions import create_langchain_embedding
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

ef = create_langchain_embedding(model_config["langchain_embeddings"])
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
chunked_docs = text_splitter.split_documents(pdf_loaded_data)

model_config["vector_db_client"].delete_collection(vector_db_collection_name)
model_config["vector_db_client"].create_collection(
    vector_db_collection_name, embedding_function=ef
)

vector_store_from_client = Chroma(
    client=model_config["vector_db_client"],
    collection_name=vector_db_collection_name,
    embedding_function=ef,
)

vector_store_from_client.add_documents(chunked_docs)

['75be8fb0-6c6a-4c16-bb4d-400c475bf8b5',
 '96a4fb5f-54d8-4900-ba22-fae27051587c',
 '893d5cf2-77a0-4f71-b120-41f4aca47681']

In [11]:
from IPython.display import Markdown, display


def my_display(display_text):
    display(Markdown(f"<b>{display_text}</b>"))

In [13]:
# validating the created index
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from chromadb.utils.embedding_functions import create_langchain_embedding


# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")
ef = create_langchain_embedding(model_config["langchain_embeddings"])

vector_db = Chroma(
    client=model_config["vector_db_client"],
    collection_name=vector_db_collection_name,
    embedding_function=ef,
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vector_db.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | model_config["llm_instance"]
    | StrOutputParser()
)

query = "About who this document speaks?"
response = qa_chain.invoke(query)

my_display(response)

/Users/rsukumar/my_workspace/job_coverletter_gen/.venv/lib/python3.12/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


<b>The document speaks about Sukumar Raghavan, a seasoned Machine Learning Engineer with 6+ years of experience in AI and over a decade of software development experience.</b>

In [21]:
# addtional context enrichment input using the job description
job_description = """
"""

In [23]:
from operator import itemgetter

from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from chromadb.utils.embedding_functions import create_langchain_embedding


def preprocess_job_description(job_description):
    """Extract key requirements and skills from job description"""
    # You could use an LLM to extract key requirements
    extraction_prompt = """
    Analyze the following job description and extract:
    1. Required technical skills
    2. Required soft skills
    3. Main responsibilities
    4. Key technologies mentioned
    
    Job Description:
    {job_description}
    
    Provide a structured summary of the key requirements.
    """

    llm = model_config["llm_instance"]
    requirements = llm.invoke(extraction_prompt.format(job_description=job_description))

    return requirements


def get_relevant_experience(vector_db, job_description):
    # First, extract key requirements from job description
    job_query = f"Find experience and skills related to: {job_description}"

    # Get relevant experience matching job requirements
    relevant_docs = vector_db.similarity_search(
        job_query,
        k=4,  # Increase number of relevant chunks
    )

    # Get general professional summary
    summary_docs = vector_db.similarity_search(
        "professional summary and key achievements", k=2
    )

    return relevant_docs + summary_docs


def build_context(retrieved_docs, job_description):
    cv_context = "\n".join([doc.page_content for doc in retrieved_docs])

    return {
        "context": cv_context,
        "job_description": job_description,
        "query": "Write a tailored cover letter for this specific job position as a job applicant.",
    }


# Generate cover letter
# Use in main flow
def generate_tailored_cover_letter(vector_store_client, job_description):
    # Preprocess job description to extract key requirements
    key_requirements = preprocess_job_description(job_description)
    ef = create_langchain_embedding(model_config["langchain_embeddings"])

    vector_db = Chroma(
        client=vector_store_client,
        collection_name=vector_db_collection_name,
        embedding_function=ef,
    )

    # Use these requirements to guide relevant experience retrieval
    relevant_docs = get_relevant_experience(vector_db, key_requirements)

    # Generate cover letter with explicit focus on matching requirements
    input_context = build_context(relevant_docs, job_description)

    # Add to the chain configuration
    chain_config = {
        "document_separator": "\n\n",
        "max_tokens_limit": 2000,
        "return_source_documents": True,
        "token_overlap": 200,
    }

    # Use local Ollama model to generate the cover letter.
    # llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", **llm_config)
    llm = model_config["llm_instance"]
    retriever = vector_db.as_retriever(
        search_kwargs={"k": 4},
        search_type="mmr",  # Use Maximum Marginal Relevance for better diversity
        **chain_config,
    )

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Prompt template
    prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer just say "I don't know", don't try to make up an answer.
    Context: {context}

    Job Description: {job_description}

    Question: {query}

    Answer:"""

    QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_template)

    qa_chain = (
        {
            "context": itemgetter("retrieved_context") | retriever | format_docs,
            "query": itemgetter("query"),
            "job_description": itemgetter("job_description"),
        }
        | QA_CHAIN_PROMPT
        | llm
        | StrOutputParser()
    )

    response = qa_chain.invoke(
        {
            "retrieved_context": input_context["context"],
            "job_description": input_context["job_description"],
            "query": input_context["query"],
        }
    )

    return response

In [ ]:
# %debug
generated_cover_letter = generate_tailored_cover_letter(
    model_config["vector_db_client"], job_description
)

my_display(generated_cover_letter)

In [27]:
import gradio as gr


def gradio_interface_to_generate_cover_letter(job_description):
    return generate_tailored_cover_letter(
        model_config["vector_db_client"], job_description
    )


with gr.Blocks() as demo:
    gr.Markdown(
        """
    # Tailored job cover letter generation using AI (Llama)!
    Submit a job description to generate a tailored cover letter for the job position.
    """
    )
    inp = gr.Textbox(
        label="Job Description", placeholder="Copy & Paste the job description here"
    )
    out = gr.Textbox(
        label="AI Generated Cover Letter",
        placeholder="AI generated cover letter will be here",
    )
    submit_btn = gr.Button("Generate Cover Letter Now!")
    submit_btn.click(
        fn=gradio_interface_to_generate_cover_letter,
        inputs=inp,
        outputs=out,
        api_name="gradio_interface_to_generate_cover_letter",
    )

demo.launch()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3
